# Hotel Reviews Tutorial

This tutorial uses the [Hotel-Review Datasets](https://www.cs.cmu.edu/~jiweil/html/hotel-review.html) to teach the following aspects:

* Data preparation and high-level analysis
* Langauge classification
* Sentiment analysis
* Content selection
* Review summarization using an open source language model

In [ ]:
import pandas as pd
import os
import tqdm

from zipfile import ZipFile
from pandas import DataFrame
from tqdm.notebook import tqdm
from lingua import Language, LanguageDetectorBuilder
from ollama import chat
from ollama import ChatResponse

In [ ]:
# Pandas options:
pd.options.mode.copy_on_write = True

## Data Preparation and Ingestion

Prepare the raw data and read it into a pandas dataframe.


In [ ]:
def unzip_datafiles(zip_file_path: str):
    """
    Unzip the raw data files and set the .txt extension to .json
    :param zip_file_path: The file path to the data zip file
    """
    with ZipFile(zip_file_path) as zip_file:
        zip_file.extractall(path="./resources/data/")

    extracted_txt_file = None

    if os.name == "nt":
        extracted_txt_file = zip_file_path.split("\\")[-1]
    else:
        extracted_txt_file = zip_file_path.split("/")[-1]

    extracted_txt_file = extracted_txt_file.replace(".zip", "")

    if os.path.exists(f"./resources/data/{extracted_txt_file}"):
        # Rename file extension:
        os.rename(f"./resources/data/{extracted_txt_file}", f"./resources/data/{extracted_txt_file.replace('.txt', '.json')}")


In [ ]:
# Process zip files:
unzip_datafiles("./resources/data/offering.txt.zip")
unzip_datafiles("./resources/data/review.txt.zip")

In [ ]:
# Read the Accommodation dataset:
accommodation_offerings_df = pd.read_json(path_or_buf="./resources/data/offering.json", lines=True)
accommodation_offerings_df = accommodation_offerings_df[["id", "name", "type", "hotel_class", "address"]]
accommodation_offerings_df.head()

In [ ]:
# Are we only dealing with Hotels?
hotel_count = len(accommodation_offerings_df.loc[accommodation_offerings_df["type"] == "hotel"])
print(f"Only accommodations are hotels? {hotel_count == len(accommodation_offerings_df)}")

In [ ]:
# Replace any hotels without a class rating as zero:
accommodation_offerings_df.fillna({"hotel_class": 0}, inplace=True)

In [ ]:
# Plot distribution of Hotel class types:
hotel_class_types = accommodation_offerings_df.groupby(["hotel_class"])["hotel_class"].count()
# TODO: Generate a bar graph of hotel class types:
hotel_class_types.plot()

In [ ]:
# Read the Review dataset:
accommodation_reviews_df = pd.read_json("./resources/data/review.json", lines=True)
accommodation_reviews_df.head()

In [ ]:
# How many reviews do we have?
# TODO: calculate the numbers for each of these print statements:
review_count = 0
print(f"Number of reviews: {review_count}")
# How many accommodations have reviews?
review_accommodation_count = 0
print(f"Number of accommodations with reviews: {review_accommodation_count}")
# Min, Max, and Average reviews for a given accommodation?
accommodation_review_counts = None
print(f"Minimum number of reviews for an accommodation: {}")
print(f"Maximum number of reviews for an accommodation: {}")
print(f"Average number of review for an accommodation: {} (std. {})")
# Are the accommodations without any reviews?
print(f"Are there accommodations without any reviews: {}")

In [ ]:
# Extract the ratings JSON column and re-order column order:
accommodation_reviews_normalised_df = accommodation_reviews_df.copy(deep=True)
ratings_df = pd.json_normalize(data=accommodation_reviews_df['ratings'])
accommodation_reviews_normalised_df.drop(columns=['ratings'], inplace=True)
accommodation_reviews_normalised_df = pd.concat([accommodation_reviews_normalised_df, ratings_df], axis=1)
accommodation_reviews_normalised_df = accommodation_reviews_normalised_df[["id", "offering_id", "title", "text",
                                                                           "author", "num_helpful_votes",
                                                                           "via_mobile", "service", "cleanliness",
                                                                           "value", "location", "sleep_quality",
                                                                           "overall", "date_stayed", "date"]]
accommodation_reviews_normalised_df.head()


## Compute average rating scores per Accommodation

For each accommodation calculate the average rating scores for each of the rating factors: `service`, `cleanliness`, `value`, `location`, `sleep_quality`, and `overall`

In [ ]:
def compute_average_scores(accommodation_id: int, column_name: str, review_dataset_df: DataFrame) -> float:
    """
    Utility function to compute average scores for different review criteria
    :param accommodation_id: The id of the accommodation offering
    :param column_name: The review criteria in question.
    :param review_dataset_df: The set of accommodation reviews DataFrame
    :return: Mean average score for the given review criteria
    """
    return review_dataset_df.loc[review_dataset_df["offering_id"] == accommodation_id][column_name].mean()


In [ ]:
tqdm.pandas()
accommodation_offerings_df["avg_service"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                         "service",
                                                                                                                         accommodation_reviews_normalised_df), axis=1)
accommodation_offerings_df["avg_cleanliness"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                         "cleanliness",
                                                                                                                         accommodation_reviews_normalised_df), axis=1)
accommodation_offerings_df["avg_value"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                         "value",
                                                                                                                         accommodation_reviews_normalised_df), axis=1)
accommodation_offerings_df["avg_location"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                       "location",
                                                                                                                       accommodation_reviews_normalised_df), axis=1)
accommodation_offerings_df["avg_sleep_quality"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                          "sleep_quality",
                                                                                                                          accommodation_reviews_normalised_df), axis=1)
accommodation_offerings_df["avg_overall"] = accommodation_offerings_df.progress_apply(lambda row: compute_average_scores(row["id"],
                                                                                                                           "overall",
                                                                                                                           accommodation_reviews_normalised_df), axis=1)

In [ ]:
accommodation_offerings_df.head()

## Visualise Accommodation Ratings

Create multiple scatter graphs between `hotel_class` and the different average ratings given by reviewers.

In [ ]:
# TOD0: create multiple scatter graphs that plots `hotel_clas` against each of the average rating columns:
accommodation_offerings_df.plot.scatter(subplots=True, figsize=(4, 2), x="hotel_class", y="")

With an exception for `avg_value` there does seem there is a relationship between the hotel star class and ratings...

## Language Detection

Some of the reviews are in French. Can we detect them and remove them from our dataset to simplify the review summarization? We will use [lingua-py](https://github.com/pemistahl/lingua-py), but other models are available for the same task.

In [ ]:
# List of languages to detect:
languages = [Language.ENGLISH, Language.FRENCH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

In [ ]:
def detect_review_langauge(review_title: str, review_text: str, langauge_detector) -> dict:
    """
    For a given String and langauge_detector calculate the detected languages and the confidence scores.
    :param review_title: The title of the review to calculate language detection on.
    :param review_text: The review text to calculate language detection on.
    :param langauge_detector:
    :return: dict with the detected languages and confidence scores.
    """
    detected_scores = {
        "lingua_english": 0.0,
        "lingua_french": 0.0
    }

    # Combine the title and text:
    review_complete = review_title + "\n" + review_text

    confidence_values = langauge_detector.compute_language_confidence_values(review_complete)
    for a_confidence_value in confidence_values:
        if a_confidence_value:
            if Language.ENGLISH == a_confidence_value.language:
                detected_scores["lingua_english"] = a_confidence_value.value
            elif Language.FRENCH == a_confidence_value.language:
                detected_scores["lingua_french"] = a_confidence_value.value
    return detected_scores

In [ ]:
# TODO: Process the reviews with the language detector:
accommodation_reviews_normalised_df['lingua_scores'] = accommodation_reviews_normalised_df.progress_apply()

In [ ]:
lingua_scores_df = pd.json_normalize(data=accommodation_reviews_normalised_df['lingua_scores'])
accommodation_reviews_languages_df = pd.concat([accommodation_reviews_normalised_df, lingua_scores_df], axis=1)
accommodation_reviews_languages_df.drop(columns="lingua_scores", inplace=True)
accommodation_reviews_languages_df.head()

In [ ]:
# TODO: Create sub-sets of English and French reviews with a confidence score >= 0.9:
accommodation_reviews_english_df = accommodation_reviews_languages_df.query("")
accommodation_reviews_french_df = accommodation_reviews_languages_df.query("")

In [ ]:
accommodation_reviews_french_df.head()

In [ ]:
accommodation_reviews_english_df.head()

In [ ]:
# TODO: What reviews have neither a high English or French confidence score?
accommodation_reviews_neither_df = accommodation_reviews_languages_df.query("")

In [ ]:
accommodation_reviews_neither_df.head()

In [ ]:
# TODO: Calculate dataset sizes:
print(f"English review dataset: {}")
print(f"French review dataset: {}")
print(f"Neither English or French review dataset: {}")

 ## Future Work:

* Explore different models and how they compare to `py-lingua` in terms of language identification such as the [xlm-roberta-base-language-detection](https://huggingface.co/papluca/xlm-roberta-base-language-detection) model on HuggingFace.
* Do users who write reviews in different languages have different ratings for the same given accommodation?

## English Review Length & Sentiment Analysis

Let's focus for now on the English reviews now and understand the types of reviews that we have. In particular, how long are the reviews and what is the user sentiment in the reviews written. Does the sentiment correlate with the review scores given?

In [ ]:
# Setup NLTK:
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt_tab')
nltk.download("popular")

In [ ]:
# TODO: Compute review text token lengths:
accommodation_reviews_english_df["review_length"] = accommodation_reviews_english_df.progress_apply()
accommodation_reviews_english_df.head()

In [ ]:
# TODO: Use the pre-trained VADER (Valence Aware Dictionary and sEntiment Reasoner) to classify the reviews:
sentiment_analyzer = SentimentIntensityAnalyzer()
accommodation_reviews_english_df["sentiment_scores"] = accommodation_reviews_english_df.progress_apply()
accommodation_reviews_english_df.head()

In [ ]:
sentiment_scores_df = pd.json_normalize(data=accommodation_reviews_english_df['sentiment_scores'])
accommodation_reviews_english_df = pd.concat([accommodation_reviews_english_df, sentiment_scores_df], axis=1)
accommodation_reviews_english_df.drop(columns="sentiment_scores", inplace=True)
accommodation_reviews_english_df.head()

In [ ]:
# Save intermediate data to CSV:
accommodation_reviews_english_df.to_csv("./resources/data/intermediate/accommodation_reviews_english.csv", index=False)

In [ ]:
# Load data [Optional]:
accommodation_reviews_english_df = pd.read_csv("./resources/data/intermediate/accommodation_reviews_english.csv")

In [ ]:
# TODO: What are the top-10 most positive reviews?

In [ ]:
# TODO: Is there any relationship between review length and positive sentiment:
accommodation_reviews_english_df.plot.scatter()

In [ ]:
# TODO: Calculate for `review_length`, `neg`, `neu`, `pos`, and `compund` average values on a per-accommodation level:
accommodation_offerings_df["avg_review_length"] = accommodation_offerings_df.progress_apply()
accommodation_offerings_df["avg_neg"] = accommodation_offerings_df.progress_apply()
accommodation_offerings_df["avg_neu"] = accommodation_offerings_df.progress_apply()
accommodation_offerings_df["avg_pos"] = accommodation_offerings_df.progress_apply()
accommodation_offerings_df["avg_compound"] = accommodation_offerings_df.progress_apply()
accommodation_offerings_df.head()

In [ ]:
# TODO: Generate scatter plots for each average aspect calculated above against `hotel_class`
accommodation_offerings_df.plot.scatter(subplots=True, figsize=(4, 2), x="hotel_class", y="")

## Review Summarization

From the above we can select the most positive five reviews for each accommodation and prompt a model to summarise the given reviews with a focus on the most relevant points. Consider experimenting with the prompt and what reviews should be given to model for summarization. In addition, the default prompt is a zero shot prompt. Would examples in the prompt help? Try using an LLM to generate the prompt for you.

Before running this section ensure that ollama is up by running: `ollama serve`

For this section we will use [Llama 3.1-8b](https://ollama.com/library/llama3.1) as our model of choice. Feel free to experiment with other models.

In [ ]:
def generate_ollama_response(content: str, language_model="llama3.1") -> str:
    response: ChatResponse = chat(model=language_model, messages=[
        {
            'role': 'user',
            'content': content,
        },
    ])
    return response['message']['content']

In [ ]:
print(generate_ollama_response("Hello!"))

In [ ]:
# Load the default prompt -- This prompt was generated by Llama 3.1:
default_prompt_template = open("./resources/data/prompt/default_prompt.txt").read()

In [ ]:
def generate_positive_review_summary(prompt_template: str, offering_id: int, reviews_dataset_df: DataFrame) -> str:
    # Get all reviews for the given accommodation ID:
    accommodation_reviews_df = reviews_dataset_df.loc[reviews_dataset_df["offering_id"] == offering_id]
    accommodation_reviews_df = accommodation_reviews_df.sort_values(by=["pos"], ascending=False).head(5)
    accommodation_reviews_list = accommodation_reviews_df["text"].tolist()
    prompt_rendered = prompt_template
    for i in range(0, len(accommodation_reviews_list)):
        prompt_rendered = prompt_rendered.replace("{{ review_" + str(i) + " }}", accommodation_reviews_list[i])
    return generate_ollama_response(prompt_rendered)

In [ ]:
# Select a random hotel and generate some review summaries -- Speed will depend on compute power
# e.g. GPU, CPU, and RAM for inferencing (reference: 2017 Macbook Pro - CPU: 2.8Ghz Quad-core i7 (Kabylake), 16GB RAM: ~5 minutes)
# Supported GPUs can be found here: https://ollama.readthedocs.io/en/gpu/
random_accommodations_df = accommodation_offerings_df.sample(n=1)
random_accommodations_df["review_summaries"] = random_accommodations_df.progress_apply(lambda row: generate_positive_review_summary(
    prompt_template=default_prompt_template,
    offering_id=row["id"],
    reviews_dataset_df=accommodation_reviews_english_df
), axis=1)
random_accommodations_df.head()

In [ ]:
print(random_accommodations_df.iloc[0]["review_summaries"])

## Conclusion

In this tutorial we have explored the end-to-end process from data analysis, enrichment, and summarization. For additional challenges consider what changes you would make in terms of:

* What additional analyses would do to the data that was not done in this notebook?
* What other data enrichment would you consider for the base dataset (e.g. external geospatial data, part-of-speech tagging, etc.)?
* How would change the way the model is prompted to generate the summaries from the reviews?
* In terms of non-English languages, what approaches would you consider?
* Evaluation: How would evaluate the accuracy and appropriateness of the text generated by the model?